 研究利用 src/agent/tools/patientinfo.py 操作患者信息，但是在引入checkpoint 是否还可以保留上次的输入

In [1]:
import os
from typing import Annotated, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver # <--- 关键组件
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from typing_extensions import TypedDict
from langgraph.prebuilt import ToolNode, tools_condition

# 如果你在 notebook 同级目录有 src 文件夹，可以这样导入
import sys
from pathlib import Path
sys.path.insert(0, str("/Users/Apple/Documents/rare-diagnosis-agent/src/multi_agent/src/agent/tools"))
from patientinfo import set_base_info, upsert_patient_facts, delete_patient_facts, patient_info_to_text

# 也可以直接把之前的 Tool 函数定义粘贴在这里...
# (为了演示，假设工具已经定义好了，放在 tools 列表中)
tools = [set_base_info, upsert_patient_facts, delete_patient_facts, patient_info_to_text]

In [2]:
class AgentState(TypedDict):
    # add_messages: 自动处理消息的追加历史
    messages: Annotated[List[BaseMessage], add_messages]
    
    # patient_info: 我们自定义的结构，默认覆盖(Overwrite)即可，
    # 因为我们的 Tool 返回的是 Command(update=...)，会自动处理合并逻辑
    patient_info: Dict[str, Any]

In [ ]:
llm = ChatOpenAI(model="mimo-v2-flash", 
                api_key="sk-c01l9de7awq589w1b0oxtwo7mzizvi487jvtaio5lbg8sabb",
                base_url="https://api.xiaomimimo.com/v1",
                temperature=0.4)

In [9]:
SYSTEM_PROMPT = """你是一个专业的**临床数据结构化引擎**。你的唯一任务是维护和更新 `patient_info` 状态。

### 1. 核心数据架构
你必须将患者信息严格映射到以下字段：

**A. 基础信息 (`base_info`)**
- 仅包含：姓名、性别、年龄、主诉 (Chief Complaint)。
- 工具：`set_base_info`

**B. 事实图谱 (`patient_info` 的子列表)**
- 工具：`upsert_patient_facts`
- **Buckets 分类标准**：
  - `symptoms`: 症状（主观感受，如头痛、胸闷）。字段：severity, duration, nature。
  - `vitals`: 生命体征（客观数值，如血压、体温、心率）。字段：value, unit, time。
  - `exams`: 检查检验（CT、血常规）。
  - `medications`: 药物使用。
  - `family_history`: 家族史。
  - `others`: 过敏史、生活习惯等。

### 2. 必须遵守的“增删改”逻辑 (CRUD Protocol)

**规则一：格式绝对严格**
- `upsert_patient_facts` 的参数 `payload` 必须是 **字典**，不能是列表！
- ❌ 错误：`payload=[{'bucket': 'symptoms', ...}]`
- ✅ 正确：`payload={'symptoms': [{'name': '头痛', ...}], 'vitals': [...]}`

**规则二：ID 管理（防止重复）**
- **新增模式**：当通过上下文判断这是新出现的症状/体征时，**绝对不要**编造 `id` 字段。系统会自动生成。
- **更新模式**：当用户修正或补充已有信息（如“刚才说的头痛其实是很痛”）时：
    1. 先观察当前的 `patient_info`。
    2. 找到对应条目（例如 "头痛"）的 `id`（例如 "XY"）。
    3. 调用工具时**必须**带上 `id="XY"`。
    4. 未提及的字段不要覆盖，只传需要更新的字段。

**规则三：推断与客观性**
- 如果用户说“血压高”，但没说数值，记录 `symptoms` -> `name="高血压症状"`。
- 只有用户提供了数值（如 "140/90"），才记录 `vitals` -> `type="血压", value="140/90"`。

### 3. 思考链 (Chain of Thought)
在行动前，进行以下内心独白：
1. **Analyze**: 用户提供了什么新信息？
2. **Check**: 这些信息在当前的 `patient_info` 里是否存在？
3. **Decide**: 
   - 存在 -> 提取 ID -> 准备更新 (Update)。
   - 不存在 -> 准备新增 (Insert)。
4. **Format**: 构造符合 Schema 的 JSON payload。

### 4. 示例 (Few-Shot)

**场景 1：新增**
用户："我发烧 39度。"
Agent 动作：
upsert_patient_facts(payload={
    "symptoms": [{"name": "发热", "temperature": "39度"}] 
})

**场景 2：更新 (假设当前已有 ID='AB' 的发热记录)**
用户："刚才量的 39 度是腋下温度。"
Agent 动作：
upsert_patient_facts(payload={
    "symptoms": [{"id": "AB", "note": "腋下温度"}] 
})
"""

In [10]:
memory = MemorySaver()
agent = create_agent(
    model=llm,
    tools=tools,
    state_schema=AgentState,
    system_prompt=SYSTEM_PROMPT,
    checkpointer=memory
)

In [11]:
config = {"configurable": {"thread_id": "patient_1001"}}

print("=== Round 1: 第一次交互 (必须初始化结构) ===")

# 输入：因为是第一次，Memory 里没数据，所以必须给 patient_info 一个空架子
input_1 = {
    "messages": [HumanMessage(content="你好，患者叫李四，45岁，胸闷。")],
    "patient_info": { 
        "base_info": {}, "symptoms": [], "vitals": [], "exams": [], 
        "medications": [], "family_history": [], "others": []
    }
}

# 必须带上 config
final_state_1 = agent.invoke(input_1, config=config)

print("R1 结果:")
print(patient_info_to_text.invoke({"state": final_state_1}))


print("\n=== Round 2: 第二次交互 (不需要初始化！) ===")

# 输入：注意看！这里只有 messages，没有 patient_info
# LangGraph 会根据 config 里的 thread_id 去自动加载上次的 patient_info
input_2 = {
    "messages": [HumanMessage(content="他还说有点头晕。")]
}

# 只要 config 里的 thread_id 一样，它就能接上
final_state_2 = agent.invoke(input_2, config=config)

print("R2 结果 (应该包含李四+胸闷+头晕):")
print(patient_info_to_text.invoke({"state": final_state_2}))

=== Round 1: 第一次交互 (必须初始化结构) ===
R1 结果:
【base_info】
- 姓名: 李四
- 年龄: 45岁
- 主诉: 胸闷

=== Round 2: 第二次交互 (不需要初始化！) ===
R2 结果 (应该包含李四+胸闷+头晕):
【base_info】
- 姓名: 李四
- 年龄: 45岁
- 主诉: 胸闷

【symptoms】
- [ID: A6] name=头晕


In [35]:
import os
from typing import Dict, Any, List, Annotated
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import AgentMiddleware, hook_config
from langgraph.graph.message import add_messages
from langgraph.runtime import Runtime
from langgraph.types import Command # 导入 Command 用于更新状态
from langchain_core.tools import tool,InjectedToolCallId

# 1. 定义包含业务字段的自定义状态
class CustomState(AgentState):
    messages: Annotated[List[BaseMessage], add_messages]
    patient_info: Dict[str, Any]

# 2. 定义工具：一个普通工具，一个用于更改状态的工具
@tool
def check_medication(name: str):
    """检查某种药物是否可用。"""
    return f"药物 {name} 目前库存充足。"

@tool
def update_patient_status(
    new_status: str, 
    tool_call_id: Annotated[str, InjectedToolCallId] # 自动注入当前的 Call ID
):
    """更新患者的最新状态信息。"""
    print(f">>> 正在更新状态为: {new_status}")
    
    # 构造必须的回复消息
    res_msg = ToolMessage(
        content=f"成功将状态更新为: {new_status}", 
        tool_call_id=tool_call_id
    )
    
    # 同时更新消息历史和自定义字段
    return Command(
        update={
            "messages": [res_msg], 
            "patient_info": {"最新状态": new_status, "姓名": "张三"}
        }
    )
# 3. 编写中间件（加入日志以观察执行频率）
class PatientContextPlugin(AgentMiddleware):
    def __init__(self, max_messages: int = 50):
        super().__init__()
        self.max_messages = max_messages

    @hook_config()
    def before_agent(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        # --- 核心验证点：加入 print 日志 ---
        print("\n>>> [CRITICAL] before_agent 钩子正在执行！(仅在启动时运行一次)")
        print(f"before_agent")
        return {
            "messages": [SystemMessage(f"当前患者背景: {state['patient_info']}")],
        }

# 4. 初始化 Agent
model = ChatOpenAI(
    model="mimo-v2-flash", 
    api_key="sk-c01l9de7awq589w1b0oxtwo7mzizvi487jvtaio5lbg8sabb",
    base_url="https://api.xiaomimimo.com/v1",
    temperature=0.4
)

agent = create_agent(
    model=model,
    tools=[check_medication, update_patient_status], # 注册工具
    system_prompt="你是一个专业的医疗助手。",
    middleware=[PatientContextPlugin()],
    state_schema=CustomState,
    checkpointer=memory
)

# 5. 测试：通过一个复杂任务强制 Agent 进行多次工具调用
initial_input = {
    "messages": [HumanMessage(content="先更新患者状态为'已入院'，然后再帮我查一下阿司匹林库存。")],
    "patient_info": {"姓名": "张三", "病史": "高血压"}
}

# 观察控制台输出
for chunk in agent.stream(initial_input, stream_mode="updates",config={"configurable": {"thread_id": "patient_2002"}}):
    for node_name, update in chunk.items():
        print(f"\n--- 节点 [{node_name}] 执行完毕 ---")
        print("状态更新:", update)


>>> [CRITICAL] before_agent 钩子正在执行！(仅在启动时运行一次)
before_agent

--- 节点 [PatientContextPlugin.before_agent] 执行完毕 ---
状态更新: {'messages': [SystemMessage(content="当前患者背景: {'姓名': '张三', '病史': '高血压'}", additional_kwargs={}, response_metadata={}, id='7f039ac1-d81e-445f-8c33-409dd54840be')]}

--- 节点 [model] 执行完毕 ---
状态更新: {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 366, 'total_tokens': 415, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'mimo-v2-flash', 'system_fingerprint': None, 'id': '2813afa44ac041aa82fd1f2e7cc4c0c1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b345e-7051-75a0-b9d0-d304e15f7e29-0', tool_calls=[{'name': 'update_patient_status', 'args': {'new_status': '已入院'}, 'id': 'call_8d1f4f4ccd

In [36]:
agent.get_state(config={"configurable": {"thread_id": "patient_2002"}})

StateSnapshot(values={'messages': [HumanMessage(content="先更新患者状态为'已入院'，然后再帮我查一下阿司匹林库存。", additional_kwargs={}, response_metadata={}, id='b1bb7e6d-09d4-4cfe-800b-c6f454782fb0'), SystemMessage(content="当前患者背景: {'姓名': '张三', '病史': '高血压'}", additional_kwargs={}, response_metadata={}, id='7f039ac1-d81e-445f-8c33-409dd54840be'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 366, 'total_tokens': 415, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'mimo-v2-flash', 'system_fingerprint': None, 'id': '2813afa44ac041aa82fd1f2e7cc4c0c1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b345e-7051-75a0-b9d0-d304e15f7e29-0', tool_calls=[{'name': 'update_patient_status', 'args': {'new_status': '已入院'}, 'id': 'call_8d1f4f4ccdc64